### Imports

In [3]:
# !pip install transformers
# !pip install SentencePiece
# !pip install transformers
import os
import sys
import pandas as pd
from transformers import pipeline
from transformers import pipeline, set_seed
from transformers import T5Tokenizer, TFT5Model, TFT5ForConditionalGeneration
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel

2022-11-16 06:46:24.224962: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-16 06:46:24.225007: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Load Data

In [5]:
my_root = "/home/jbobro/final_project/W266_Final_Project"

In [6]:
def pull_data(df_name):
    return pd.read_parquet(f"{my_root}/Processed_Data/{df_name}.parquet.gzip")

In [7]:
df_abstract = pull_data("df_abstract")
df_author = pull_data("df_author")
df_title = pull_data("df_title")
df_fid = pull_data("df_fid")
df_bow = pull_data("df_bow")

# Text Generation

Text generation models are those where an input is given, and the model generates the next sequence that is contextually and gramatically correct. We propose that, given our collection of succesful abstracts, we can train such a model (potentially on top of an already pre-trained model) to generate a succesful abstract that may be approved by the NSF. 


#### Common Metrics of evaluation:
  - Cross Entropy Loss
  - Perplexity

## GPT-2 Text Generation Model - Decoder Only
Can we generate a meaningful abstract given a sentence or two? 

1. Identify evaluation task (GLUE) 
2. Determine pre-training approaches
3. Determine pre-training data sets 
4. Determine fine-tuning approach

In [7]:
generator = pipeline('text-generation', model='gpt2')
set_seed(42)
# generator("Hello, I'm a language model,", max_length=30, num_return_sequences=5)
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt2_model = TFGPT2LMHeadModel.from_pretrained('gpt2')

2022-11-16 05:08:22.857792: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-16 05:08:22.857849: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-16 05:08:22.857875: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (instance-3): /proc/driver/nvidia/version does not exist
2022-11-16 05:08:22.884273: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-16 05:08:23.212776: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, b

In [8]:
abs_sample = df_abstract['abstract'][1]
first_sentence = abs_sample[20:397]

In [12]:
text_start = 'Excavations by Dr. Alan Simmons and his colleagues at the site ofAkrotiri-Aetokremnos, located on the Akrotiri Peninsula ofsouthern Cyprus, have greatly surprised both archaeologists andbiologists, because they seem to indicate the co-occurrence ofhumans and pygmy forms of both hippopotamus and elephant '
input_ids = gpt2_tokenizer.encode(text_start, return_tensors='tf')
input_ids

<tf.Tensor: shape=(1, 71), dtype=int32, numpy=
array([[40127,   615,   602,   416,  1583,    13, 12246, 22553,   290,
          465,  7810,   379,   262,  2524,   286, 33901, 10599, 14783,
           12,    32,   316,   482,  2787, 39369,    11,  5140,   319,
          262,  9084, 10599, 14783, 24078,   286,    82,  4927, 27885,
           11,   423,  9257,  6655,  1111, 38967,   290,  8482,  9251,
           11,   780,   484,  1283,   284,  7603,   262,   763,    12,
        13966, 33928,   286, 40205,   290, 12972,    70,  1820,  5107,
          286,  1111, 18568, 43372, 25509,   290, 20950,   220]],
      dtype=int32)>

In [11]:
abs_sample

'Excavations by Dr. Alan Simmons and his colleagues at the site ofAkrotiri-Aetokremnos, located on the Akrotiri Peninsula ofsouthern Cyprus, have greatly surprised both archaeologists andbiologists, because they seem to indicate the co-occurrence ofhumans and pygmy forms of both hippopotamus and elephant.  Datedto the ninth millennium BC, this stratified cave site containsboth stone tools as well as the broken bones of both species.These data also indicate that human settlement of the easternMediterranean islands occurred ca. 2,000 years earlier thanpreviously expected.  Evidence seems to indicate that a trueassociation exists between the bones and stone tools.  Thisimplies human hunting of these species and a probable human rolein their extinction.  In this project, Dr. Simmons and hiscolleagues will analyze the materials recovered during multipleseasons of field research.  Objectives include analysis of faunaland material culture assemblages, of site geomorphology,stratigraphy and pa

In [13]:
generated_text_outputs = gpt2_model.generate(
    input_ids, 
    max_length=300,
    num_return_sequences=1,
    no_repeat_ngram_size=3,
    repetition_penalty=1.5,
    top_p=0.92,
    temperature=.85,
    do_sample=True,
    top_k=125,
    early_stopping=True
)

#Clearly the raw GPT-2 model is not doing too well at predicting Scientific abstracts
for i, beam in enumerate(generated_text_outputs):
  print()
  print("{}: {}".format(i, gpt2_tokenizer.decode(beam, skip_special_tokens=True)))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence



0: Excavations by Dr. Alan Simmons and his colleagues at the site ofAkrotiri-Aetokremnos, located on the Akrotiri Peninsula ofsouthern Cyprus, have greatly surprised both archaeologists andbiologists, because they seem to indicate the co-occurrence ofhumans and pygmy forms of both hippopotamus and elephant ichthyosaurs in our own day - a discovery that has led many researchers looking for some kind's proof as well as an explanation about how their ancestors evolved from those bipedal animals!
If you want more details then see... A link is posted between these three accounts: http://www'thenkionaelden/aeon105057011143_2x2049371430861/-fv3e60ea0cf4b42 https:/ / twitter & facebook The following text was first published here under permission granted [with attribution]


## GPT-3

"While Generative Pre-Trained Transformers are a great milestone in the artificial intelligence race, it’s not equipped to handle complex and long language formations. If you imagine a sentence or paragraph that contains words from very specialized fields such as literature, finance or medicine, for example, the model would not be able to generate appropriate responses without sufficient training beforehand."
  - This means without sufficient compute GPT-3 and GPT-2 would be difficult for us to train
  - It will most likely not give us as topic-specific accurate results (more focused on language structure than niche corpus) 


In [ ]:
generator = pipeline('text-generation', model='EleutherAI/gpt-neo-1.3B')

In [ ]:
abs_sample = df_abstract['abstract'][3]
first_sentence = abs_sample[20:397]

In [ ]:
text = 'Who is Elon Musk'
abs_sample = df_abstract['abstract'][1
result = generator(text, max_length=100, do_sample=True, temperature=0.9)
result

## T5 model for text generation
*Transformer: type of neural network architectu*

How does T5 work? 
- T5 is trained on C4 -- Colossal Clean Crawled Corpus (obtained by scraping web pages and ignoring the markup from the HTML, hence causing lots of gibberish text as we see bellow, to flow through to the model 


**Reference Abstract: (Not fed into the model)**

"Recent advances in our understanding of minor biologicalfractionation of Ge/Si by oceanic diatoms suggest that Ge/Si)opal  variationsmeasured in late Pleistocene piston cores and Cenozoic  drill cores arerecording whole ocean (Ge/Si)seawater variations  driven by rapid and largeglacial to interglacial changes in  continental weathering intensity and riverfluxes to the sea. If  so, then it is clearly important to produce highresolution records  of (Ge/Si)opal in cores with well-established  18 O and 14Cstratigraphies across transitions to establish the shape and timing  of theoceanic Ge/Si response.." 

**Input Sentence:**

" Recent advances in our understanding of minor biologicalfractionation of Ge/Si by oceanic diatoms suggest that Ge/Si)opal  variationsmeasured in late Pleistocene piston cores and Cenozoic  drill cores arerecording whole ocean (Ge/Si)seawater variations  driven by rapid and largeglacial to interglacial changes in  continental weathering intensity and riverfluxes to the sea."

**Pre-Trained T5 model summarization**

"this project aims to produce high resolution records of (Ge/Si)opal in cores with well-established 18 O and 14Cstratigraphies across transitions . it is also necessary to carefully evaluate and remove the effects of ge/si fractionation in local records, says dr. j. floelich. this is aimed at accomplishingtwo major goals: (1) Produce highresolution records incores that contain abundant diatoms and forams in collaboration with others' measurements a -"

Notice the previously unseen output that matches between the input sentence, and the predicted output: 
- Original: If  so, then it is clearly important to produce highresolution records  of (Ge/Si)opal ..
- Predicted: Produce highresolution records incores that contain abundant diatoms and forams in collaboration with others' measurements



In [ ]:
class TranslationDataGenerator(tf.keras.utils.Sequence):
    
    def __init__(self,
                 tokenizer,
                 model,
                 n_examples,
                 data_filename,
                 max_length=128,
                 batch_size=16,
                 shuffle=True):
        
        self.tokenizer = tokenizer
        self.model = model
        self.n_examples = n_examples
        self.data_filename = data_filename
        self.max_length = max_length
        self.batch_size = batch_size
        self.shuffle = shuffle
        
        # Initialize row order, call on_epoch_end to shuffle row indices
        self.row_order = np.arange(1, self.n_examples+1)
        self.on_epoch_end()
    
    def __len__(self):
        return self.n_examples // self.batch_size
    
    def __getitem__(self, idx):
        batch_start = idx * self.batch_size
        batch_end = (idx + 1) * self.batch_size

        # Indices to skip are the ones in the shuffled row_order before and
        # after the chunk we'll use for this batch
        batch_idx_skip = self.row_order[:batch_start] + self.row_order[batch_end:]
        df = pd.read_csv(self.data_filename, skiprows=batch_idx_skip)
        
        text_pairs = df[['orig', 'target']].values.astype(str).tolist()
        
        batch_data = preprocess_data(
            text_pairs,
            self.tokenizer,
            self.model,
            self.max_length
        )

        return batch_data
    
    def __call__(self):
        for i in range(self.__len__()):
            yield self.__getitem__(i)
            
            if i == self.__len__()-1:
                self.on_epoch_end()
    
    def on_epoch_end(self):
        if self.shuffle:
            self.row_order = list(np.random.permutation(self.row_order))

In [4]:
model_name = 't5-base'
t5_tokenizer = T5Tokenizer.from_pretrained(model_name)
t5_model = TFT5ForConditionalGeneration.from_pretrained(model_name)
# t5_model = TFT5ForConditionalGeneration.from_pretrained('t5-small')
# t5_tokenizer = T5Tokenizer.from_pretrained('t5-small')
# t5_model.summary()

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

/home/jbobro/anaconda3/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5.py:164: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

2022-11-16 06:46:59.188441: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-16 06:46:59.188491: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-16 06:46:59.188514: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (instance-3): /proc/driver/nvidia/version does not exist
2022-11-16 06:46:59.214537: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-16 06:46:59.529066: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, b

In [8]:
abs_sample = df_abstract['abstract'][3]
first_sentence = abs_sample[0:205]
first_sentence

'Professor Hull is engaged in an investigation of two separateprojects:  the first concerns criteria for selection of taxonomicprinciples; the second concerns the processes by which scienceitself develops. '

In [9]:
abs_sample

'Professor Hull is engaged in an investigation of two separateprojects:  the first concerns criteria for selection of taxonomicprinciples; the second concerns the processes by which scienceitself develops.  While seemingly unrelated, Professor Hull isusing the first study as a case for exemplifying his argumentsconcerning the processes of scientific development.Although the distance between alternative principles of biologicalclassification and anything that might be considered "evidence" forthem is great, empirical considerations can be brought to bear onthe decision to choose one set of principles of classification overothers.  Professor Hull argues that the taxonomic principles thatresult in classifications that facilitate the discovery of naturalregularities are preferable to those that do not facilitate suchdiscoveries.  Dr Hull is examining the recent literature inevolutionary biology to see what sorts of classification aid eitherphylogenetic reconstruction or the discovery of re

In [10]:
ARTICLE = first_sentence
t5_input_text = "summarize: " + abs_sample
t5_inputs = t5_tokenizer([t5_input_text], return_tensors='tf')

In [ ]:
t5_summary_ids = t5_model.generate(t5_inputs['input_ids'],
                                   num_beams=7, 
                                   no_repeat_ngram_size=2, 
                                   min_length=400, 
                                   max_length=600)

print([t5_tokenizer.decode(g, skip_special_tokens=True,
                           clean_up_tokenization_spaces=False)
       for g in t5_summary_ids])

2022-11-16 06:48:59.453579: I tensorflow/compiler/xla/service/service.cc:171] XLA service 0x55d1b9819d50 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-11-16 06:48:59.453618: I tensorflow/compiler/xla/service/service.cc:179]   StreamExecutor device (0): Host, Default Version
2022-11-16 06:49:00.483488: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:237] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2022-11-16 06:49:02.012898: I tensorflow/compiler/jit/xla_compilation_cache.cc:399] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
